In [4]:
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [5]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [6]:
test_labels.shape

(10000,)

In [7]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32,3])
    
    
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv1_bn = tf.layers.batch_normalization(
        inputs=conv1,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv1_bn_relu = tf.nn.relu(conv1_bn)
    
    conv2 = tf.layers.conv2d(
      inputs=conv1_bn_relu,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv2_bn = tf.layers.batch_normalization(
        inputs=conv2,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv2_bn_relu = tf.nn.relu(conv2_bn)
    pool1 = tf.layers.max_pooling2d(inputs=conv2_bn_relu, pool_size=[2, 2], strides=2)
    
    dropout1 = tf.layers.dropout(
      inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)


    
    
    conv3 = tf.layers.conv2d(
      inputs=dropout1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv3_bn = tf.layers.batch_normalization(
        inputs=conv3,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv3_bn_relu = tf.nn.relu(conv3_bn)
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn_relu,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv4_bn = tf.layers.batch_normalization(
        inputs=conv4,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv4_bn_relu = tf.nn.relu(conv4_bn)
    pool2 = tf.layers.max_pooling2d(inputs=conv4_bn_relu, pool_size=[2, 2], strides=2)
    
    
    dropout2 = tf.layers.dropout(
      inputs=pool2, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    
    conv5 = tf.layers.conv2d(
      inputs=dropout2,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv5_bn = tf.layers.batch_normalization(
        inputs=conv5,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv5_bn_relu = tf.nn.relu(conv5_bn)
    
    conv6 = tf.layers.conv2d(
      inputs=conv5_bn_relu,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv6_bn = tf.layers.batch_normalization(
        inputs=conv6,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv6_bn_relu = tf.nn.relu(conv6_bn)
    pool3 = tf.layers.max_pooling2d(inputs=conv6_bn_relu, pool_size=[2, 2], strides=2)
    
    
    dropout3 = tf.layers.dropout(
      inputs=pool3, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    flattened = tf.reshape(dropout3, [-1, 4 * 4 * 128])
    dense1 = tf.layers.dense(inputs=flattened, units=2048, activation=None)
    logits = tf.layers.dense(inputs=dense1, units=10)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits/temperature, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [8]:
def cross_entropy2(tar_soft_t, pred_logits):
   pred_soft_t = tf.nn.softmax(pred_logits/temperature)
   pred_log = -1 * tf.log(pred_soft_t)
   product =tf.multiply(tar_soft_t,pred_log)
   return tf.reduce_mean(product)
    
def custom_loss(y_true, pred_logits, tar_soft_t):
  return  loss_weight * tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits)+(1-loss_weight)*cross_entropy2(tar_soft_t,pred_logits)


In [9]:
def getFilterData(f,l):
  sess=tf.InteractiveSession()
  tf.train.start_queue_runners(sess)
  data_s=f['x'].eval()
  out_s=l.eval()
  sess.close()
  return data_s,out_s

In [17]:

def student_model_fn(features, labels, mode):

    data_swap,out_swap=getFilterData(features,labels)
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    if mode == tf.estimator.ModeKeys.TRAIN:
        eval_teacher_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x":data_swap},
          y=out_swap,
          batch_size=100,
          shuffle=False)
        eval_teacher=cifar_classifier.evaluate(input_fn=eval_teacher_fn)
        outlog.write('%f;' % eval_teacher['accuracy'])
        outlog.write('%f;' % eval_teacher['loss'])
        outlog.write('\n')
        predictions=cifar_classifier.predict(input_fn=eval_teacher_fn)
        teacher_pred=list(predictions)
        teacher_soft=[ p['probabilities'] for p in teacher_pred]
        temp=np.array(teacher_soft)

        teacher_soft_t=tf.convert_to_tensor(temp)




    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=8,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv1_bn = tf.layers.batch_normalization(
        inputs=conv1,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
        )
    conv1_bn_relu = tf.nn.relu(conv1_bn)

    conv2 = tf.layers.conv2d(
        inputs=conv1_bn_relu,
        filters=8,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv2_bn = tf.layers.batch_normalization(
        inputs=conv2,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        )
    conv2_bn_relu = tf.nn.relu(conv2_bn)
    pool1 = tf.layers.max_pooling2d(inputs=conv2_bn_relu, pool_size=[2, 2], strides=2)

    dropout1 = tf.layers.dropout(
        inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)




    conv3 = tf.layers.conv2d(
        inputs=dropout1,
        filters=16,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv3_bn = tf.layers.batch_normalization(
        inputs=conv3,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
        )
    conv3_bn_relu = tf.nn.relu(conv3_bn)

    conv4 = tf.layers.conv2d(
        inputs=conv3_bn_relu,
        filters=16,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv4_bn = tf.layers.batch_normalization(
        inputs=conv4,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        )
    conv4_bn_relu = tf.nn.relu(conv4_bn)
    pool2 = tf.layers.max_pooling2d(inputs=conv4_bn_relu, pool_size=[2, 2], strides=2)


    dropout2 = tf.layers.dropout(
        inputs=pool2, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN)


    conv5 = tf.layers.conv2d(
        inputs=dropout2,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv5_bn = tf.layers.batch_normalization(
        inputs=conv5,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
        )
    conv5_bn_relu = tf.nn.relu(conv5_bn)

    conv6 = tf.layers.conv2d(
        inputs=conv5_bn_relu,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv6_bn = tf.layers.batch_normalization(
        inputs=conv6,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        )
    conv6_bn_relu = tf.nn.relu(conv6_bn)
    pool3 = tf.layers.max_pooling2d(inputs=conv6_bn_relu, pool_size=[2, 2], strides=2)


    dropout3 = tf.layers.dropout(
        inputs=pool3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    flattened = tf.reshape(dropout3, [-1, 4 * 4 * 32])
    dense1 = tf.layers.dense(inputs=flattened, units=512, activation=None)
    logits = tf.layers.dense(inputs=dense1, units=10)

    
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss=custom_loss(labels, logits,teacher_soft_t)
    else:
        print(labels.shape,logits.shape)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)


    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
    }
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [18]:
print(train_labels.shape)

(50000,)


In [21]:

# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_model_teacher_6_conv_earlystop")

temp_list=[1,2,5,10,15,20,30]
loss_weight_list=[0,0.25,0.5,0.75,1]
p=-1
l=-1
def getdatap():
    global p
    p += 1
    if p>len(train_labels)/100:
        p=0
    data=train_data[p*100:(p+1)*100]
    labels=train_labels[p*100:(p+1)*100]
    
    datatensor=tf.convert_to_tensor(data)
    labeltensor=tf.convert_to_tensor(labels)
    
    
    return {'x':datatensor},labeltensor

out = open('cifar_temp_loss6.csv', 'w')
outlog = open('out_log.csv', 'w')



for i in temp_list:
    for j in loss_weight_list:
        temperature=i
        loss_weight=j
        # Create the Estimator
        student_classifier = tf.estimator.Estimator(
          model_fn=student_model_fn, model_dir="./models/CIFAR_SINGLE_student_t_"+str(i)+"_w_"+str(j))

        student_classifier.train(
          input_fn=getdatap,
          steps=1000,
          hooks=None)
        
        eval_train_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          shuffle=False)
        train_result=student_classifier.evaluate(input_fn=eval_train_fn)
        
        out.write('%f;' % i)
        out.write('%f;' % j)
        out.write('%f;' % train_result['accuracy'])
        out.write('%f;' % train_result['loss'])
        
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": test_data},
          y=test_labels,
          num_epochs=1,
          shuffle=False)
        eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
        # output=list(predictions)
        
        out.write('%f;' % eval_result['accuracy'])
        out.write('%f;' % eval_result['loss'])
#         acc=eval_result
#         out.write(eval_result)
        out.write('\n')
        
out.close()
outlog.close()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cifar_model_teacher_6_conv_earlystop', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E8351854E0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/CIFAR_SINGLE_student_t_1_w_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_eve

Exception in thread Thread-855:
Traceback (most recent call last):
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\estimator\inputs\queues\feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1058, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



(?,) (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-11-14:11:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/CIFAR_SINGLE_student_t_1_w_0\model.ckpt-27011
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-11-14:11:18
INFO:tensorflow:Saving dict for global step 27011: accuracy = 0.272997, global_step = 27011, loss = 2.56095
INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


Exception in thread Thread-858:
Traceback (most recent call last):
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\estimator\inputs\queues\feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1058, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



(?,) (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-11-14:11:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/CIFAR_SINGLE_student_t_1_w_0\model.ckpt-27011
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-11-14:11:21
INFO:tensorflow:Saving dict for global step 27011: accuracy = 0.273814, global_step = 27011, loss = 2.57195
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/CIFAR_SINGLE_student_t_1_w_0.25', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E85FA99

...

(?,) (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-11-14:36:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/CIFAR_SINGLE_student_t_30_w_0.5\model.ckpt-2010
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-11-14:36:29
INFO:tensorflow:Saving dict for global step 2010: accuracy = 0.231489, global_step = 2010, loss = 7.25596
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/CIFAR_SINGLE_student_t_30_w_0.75', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E8AEC8

Exception in thread Thread-1188:
Traceback (most recent call last):
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\estimator\inputs\queues\feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1058, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



(?,) (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-11-14:37:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/CIFAR_SINGLE_student_t_30_w_0.75\model.ckpt-2010
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-11-14:37:15
INFO:tensorflow:Saving dict for global step 2010: accuracy = 0.214018, global_step = 2010, loss = 8.92579
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/CIFAR_SINGLE_student_t_30_w_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E87839B0

Exception in thread Thread-1198:
Traceback (most recent call last):
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\estimator\inputs\queues\feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1058, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



(?,) (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-11-14:37:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/CIFAR_SINGLE_student_t_30_w_1\model.ckpt-2010
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-11-14:38:00
INFO:tensorflow:Saving dict for global step 2010: accuracy = 0.212458, global_step = 2010, loss = 12.6747


In [45]:

# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_model_teacher_6_conv_earlystop")

temp_list=[1,2,5,10,15,20,30]
loss_weight_list=[0,0.25,0.5,0.75,1]
p=-1
l=-1
def getdatap():
    global p
    p += 1
    if p>len(train_labels)/100:
        p=0
    data=train_data[p*100:(p+1)*100]
    labels=train_labels[p*100:(p+1)*100]
    
    datatensor=tf.convert_to_tensor(data)
    labeltensor=tf.convert_to_tensor(labels)
    
    
    return {'x':datatensor},labeltensor

out = open('cifar_temp_loss9.csv', 'w')
outlog = open('out_log.csv', 'w')


temp_list=[20]
loss_weight_list=[0.75]
for i in temp_list:
    for j in loss_weight_list:
        temperature=i
        loss_weight=j
        # Create the Estimator
        student_classifier = tf.estimator.Estimator(
          model_fn=student_model_fn, model_dir="./models/CIFAR_SINGLE_studentsssss")

        
        
        eval_train_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          batch_size=100,
          shuffle=False)
        
        student_classifier.train(
          input_fn=eval_train_fn,
          steps=2000,
          hooks=None)
        
        
        eval_train_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          shuffle=False)
        train_result=student_classifier.evaluate(input_fn=eval_train_fn)
        print(train_result)
        out.write('%f;' % i)
        out.write('%f;' % j)
        out.write('%f;' % train_result['accuracy'])
        out.write('%f;' % train_result['loss'])
        
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": test_data},
          y=test_labels,
          num_epochs=1,
          shuffle=False)
        eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
        print(eval_result)
        # output=list(predictions)
        
        out.write('%f;' % eval_result['accuracy'])
        out.write('%f;' % eval_result['loss'])
#         acc=eval_result
#         out.write(eval_result)
        out.write('\n')
        
out.close()
outlog.close()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cifar_model_teacher_6_conv_earlystop', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E876C740B8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/CIFAR_SINGLE_studentsssss', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_

Exception in thread Thread-1412:
Traceback (most recent call last):
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\estimator\inputs\queues\feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1058, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



(?,) (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-11-16:09:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/CIFAR_SINGLE_studentsssss\model.ckpt-6461
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-11-16:09:31
INFO:tensorflow:Saving dict for global step 6461: accuracy = 0.635295, global_step = 6461, loss = 1.01365
{'accuracy': 0.63529533, 'loss': 1.0136513, 'global_step': 6461}
